In [1]:
import LDA
import time
import numpy as np
import matplotlib.pyplot as plt
import collections
import json
from scipy import sparse

In [2]:
with open('CS281/temp/Edi10.json', 'r') as fp:
    reduced_edi = json.load(fp)

with open('CS281/temp/word_to_index.json', 'r') as fp:
    word_to_index = json.load(fp)

with open('CS281/temp/index_to_word.json', 'r') as fp:
    index_to_word = json.load(fp)

with open('CS281/temp/bid_to_index.json', 'r') as fp:
    bid_to_index = json.load(fp)

with open('CS281/temp/Edi10.json', 'r') as fp:
    index_to_bid = json.load(fp)

In [3]:
vocab10 = word_to_index.keys()
nonzero_data = []
rows_s = []
cols_s = []

for k in reduced_edi.keys():
    counter = collections.Counter(reduced_edi[k])
    nonzero_data += counter.values()
    rows_s += [bid_to_index[k]]*len(counter.values())
    cols_s += [word_to_index[ck] for ck in counter.keys()]

sparse_mat = sparse.csc_matrix((nonzero_data,(rows_s,cols_s)),shape = (len(bid_to_index),len(word_to_index)))

dtm_edi = sparse_mat.toarray()

In [4]:
K = [5,7,10,15]
ldas = {}
perp = {}
for k in K:
    print k
    np.random.seed(10000)
    ldas[k] = LDA.lda_minibatch(dtm_edi[:580,:],k,10,512,0.6,100)
    perp[k] = LDA.perplexity_test(ldas[k],dtm_edi[580:,],100)

best = max(perp, key=perp.get)

5
7
10
15


In [5]:
best

5

In [12]:
for k in K:
    print "Number of topics: %s" % k
    if k==best:
        print "***** BEST K *****"
    LDA.show_topics(ldas[k],range(k),10,vocab10)

Number of topics: 5
Topic 0:

[u'spoonful', u'icecream', u'punk', u'eating', u'chai', u'inconvenience', u'shoulder', u'pence', u'need', u'\xa39.95']

Topic 1:

[u'nut', u'wood', u'cheeseburger', u'baker', u'vegetarian', u'road', u'feature', u'la', u'frontage', u'purchase']

Topic 2:

[u'research', u'pil', u'friend', u'ginger', u'crust', u'crisps', u'recipe', u'toast', u'university', u'eating']

Topic 3:

[u'broccoli', u'presence', u'round', u'calory', u'hazelnut', u'heap', u'general', u'branch', u'handful', u'noise']

Topic 4:

[u'whilst', u'walk', u'fall', u'slab', u'liver', u'hang', u'feature', u'chipotle', u'travel', u'kilo']

Number of topics: 7
***** BEST K *****
Topic 0:

[u'presence', u'round', u'calory', u'heap', u'branch', u'handful', u'noise', u'dauphinoise', u'parcel', u'creature']

Topic 1:

[u'punk', u'inconvenience', u'feature', u'shoulder', u'chipotle', u'date', u'relaxing', u'kilt', u'cousin', u'artwork']

Topic 2:

[u'broccoli', u'fall', u'hang', u'slab', u'liver', u'g

In [6]:
perp2 = {}
tau = [256,512,1024]
kappa = [0.6,0.7,0.8]

ldas2 = {}
for t in tau:
    for k in kappa:
        print (t,k)
        np.random.seed(10)
        ldas2[(t,k)] = LDA.lda_minibatch(dtm_edi[:580,:],best,10,t,k,100)
        perp2[(t,k)] = LDA.perplexity_test(ldas2[(t,k)],dtm_edi[580:,],100)

(256, 0.6)
(256, 0.7)
(256, 0.8)
(512, 0.6)
(512, 0.7)
(512, 0.8)
(1024, 0.6)
(1024, 0.7)
(1024, 0.8)


In [7]:
best_combi = max(perp2, key=perp.get)

In [8]:
best_combi

(512, 0.8)

In [ ]:
BEST_EDI = LDA.lda_minibatch(dtm_edi,best,10,512,0.8,100)

In [67]:
BEST_EDI[0].shape

(2313, 6)

In [11]:
np.save("CS281/temp/edi_topics",BEST_EDI[0])

In [ ]:
np.save("CS281/temp/edi_assignment",BEST_EDI[1])

In [58]:
def clean_topics(lda,thresh1):
    topics = lda[0].copy()
    gammas = lda[1].copy()
    notflag_uni = []
    for i in range(topics.shape[1]):
        if 1 - spatial.distance.cosine(topics[:,i], [1./topics.shape[0]]*topics.shape[0])<thresh1:
            notflag_uni.append(i)
    newtopics = topics[:,notflag_uni]
    newgammas = gammas[:,notflag_uni]/gammas[:,notflag_uni].sum(axis=1)[:,np.newaxis]
    
    
    return newtopics,newgammas